In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import heapq
import itertools
import copy
import os
import seaborn as sns
import pandas as pd
from PIL import Image
import dataset_64
import time

/home/hoge/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pretrained_model = "alphabet_model64.pth" #事前学習済みMNISTモデル(重みパラメータ)
use_cuda = True

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(57600, 128)
        self.fc2 = nn.Linear(128, 26)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

# MNISTのTest datasetと dataloaderの定義
# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
#             transforms.ToTensor(),
#             transforms.Normalize((0.5,), (0.5,))
#             ])), 
#         batch_size=1, shuffle=False)

# 使うデバイス（CPUかGPUか）の定義
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

# ネットワークの初期化
model = Net().to(device)
print(model)
# 訓練済みモデルのロード
model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))


# モデルを評価モードに設定。本チュートリアルの例では、これはドロップアウト層等を評価モードにするのに必要
model.eval()

CUDA Available:  True
Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=57600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=26, bias=True)
)


Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=57600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=26, bias=True)
)

In [4]:
def im_is_black(image):
    if image < torch.Tensor([0]).to("cuda"):
        return True
    else:
        return False

In [5]:
def im_is_white(image):
    if image >= torch.Tensor([0]).to("cuda"):
        return True
    else:
        return False

In [6]:
def character_search(image):
    image_copy = copy.deepcopy(image)
    for x in range(64):
        for y in range(64):
            if im_is_black(image[x][y]):
                for i in range(-1, 2):
                    for j in range(-1, 2):
                        if i != 0 and j != 0:
                            continue
                        elif (x + i != -1) and (x + i != 64) and (y + j != -1) and (y + j != 64):
                            if im_is_white(image[x + i][y + j]):
                                image_copy[x][y] = torch.Tensor([1]).to("cuda")
                                break
                    else:
                        continue
                    break
                else:
                    image_copy[x][y] = torch.Tensor([0]).to("cuda")
            elif im_is_white(image[x][y]):
                for i in range(-1, 2):
                    for j in range(-1, 2):
                        if i != 0 and j != 0:
                            continue
                        elif (x + i != -1) and (x + i != 64) and (y + j != -1) and (y + j != 64):
                            if im_is_black(image[x + i][y + j]):
                                image_copy[x][y] = torch.Tensor([1]).to("cuda")
                                break
                    else:
                        continue
                    break
                else:
                    image_copy[x][y] = torch.Tensor([0]).to("cuda")
    image_chain = list(itertools.chain.from_iterable(image_copy))
    return image_chain

In [7]:
# def attack(data, data_grad, target, loss_old, dirname_pro, chr, count):
#     max_index_list = []
#     px = 0
    
#     data_grad_chain = list(itertools.chain.from_iterable(data_grad[0][0]))
#     character_coordinate = character_search(data.data[0][0])
#     for i in range(4096):
#         if character_coordinate[i] == torch.Tensor([0]).to("cuda"):
#             data_grad_chain[i] = torch.Tensor([0]).to("cuda")
#     for j in max_index_list:
#         data_grad_chain[j] = torch.Tensor([0]).to("cuda")
#     data_grad_chain_abs = list(map(abs, data_grad_chain))
#     all_blocks = []
#     for i in range(4):
#         for j in range(4):
#             block = []
#             for k in range(16):
#                 for l in range(16):
#                     block.append(data_grad_chain_abs[i * 1024 + j * 16 + k * 64 + l])
#             block_sum = sum(block)
#             all_blocks.append(block_sum)
    
#     block_max_id = all_blocks.index(max(all_blocks))
#     block_max_index = [block_max_id % 4, block_max_id // 4]

#     best_px = 0
#     for i in range(16):
#         for j in range(16):
#             if best_px < data_grad_chain_abs[block_max_index[0] * 16 + block_max_index[1] * 1024 + i * 64 + j]:
#                 best_px = data_grad_chain_abs[block_max_index[0] * 16 + block_max_index[1] * 1024 + i * 64 + j]
    
#     attack_list = [best_px]

#     for p in attack_list:
        
#         if px == 0:
#             data_grad_chain_abs_before = copy.deepcopy(data_grad_chain_abs)

#         for i in range(8):
#             c = max([data_grad_chain_abs[data_grad_chain_abs_before.index(p) - 65], data_grad_chain_abs[data_grad_chain_abs_before.index(p) - 64], data_grad_chain_abs[data_grad_chain_abs_before.index(p) - 63], data_grad_chain_abs[data_grad_chain_abs_before.index(p) - 1], data_grad_chain_abs[data_grad_chain_abs_before.index(p)  + 1], data_grad_chain_abs[data_grad_chain_abs_before.index(p) + 63], data_grad_chain_abs[data_grad_chain_abs_before.index(p) + 64], data_grad_chain_abs[data_grad_chain_abs_before.index(p) + 65]])
#             print(c)
#             grad_index = data_grad_chain_abs.index(c)
#             print("abcd")
        
            
#             if (data_grad_chain[grad_index] > 0 and data[0][0][grad_index // 64][grad_index - 64 * (grad_index // 64)] < 0) or (data_grad_chain[grad_index] < 0 and data[0][0][grad_index // 64][grad_index - 64 * (grad_index // 64)] > 0):
#                 data.requires_grad = False
#                 data[0][0][grad_index // 64][grad_index - 64 * (grad_index // 64)] *= -1
#                 data.requires_grad = True
#                 output = model(data)
#                 loss = F.nll_loss(output, target)
#                 if loss < loss_old:
#                     data.requires_grad = False
#                     data[0][0][grad_index // 64][grad_index - 64 * (grad_index // 64)] *= -1
#                     continue
#                 # image[0][0][grad_max_index // 64][grad_max_index - 64 * (grad_max_index // 64)] += ((image[0][0][grad_max_index // 64][grad_max_index - 64 * (grad_max_index // 64)] == torch.Tensor([-1]).to("cuda")) * 0.2 - torch.Tensor([0.1]).to("cuda")).squeeze(0)
#                 px += 1
#                 plt.xticks([], [])
#                 plt.yticks([], [])
#                 plt.imsave(dirname_pro + chr + "/{}".format(count) + "/" + "{}.png".format(px), data.squeeze().detach().cpu().numpy(), cmap="gray")
#                 print("px:{}".format(px))
#                 pred = output.max(1, keepdim=True)[1]
#                 if pred.item() != target.item():
#                     break
#                 loss_old = loss
#                 max_index_list.append(grad_index)
#                 attack_list.append(c)
#                 model.zero_grad()
#                 # 逆伝播させてモデルの勾配を計算します
#                 loss.backward()
#                 # データの勾配を取得します
#                 data_grad = data.grad.data
#                 data_grad_chain = list(itertools.chain.from_iterable(data_grad[0][0]))
#                 character_coordinate = character_search(data.data[0][0])
#                 for i in range(4096):
#                     if character_coordinate[i] == torch.Tensor([0]).to("cuda"):
#                         data_grad_chain[i] = torch.Tensor([0]).to("cuda")
#                 for j in max_index_list:
#                     data_grad_chain[j] = torch.Tensor([0]).to("cuda")
#                 data_grad_chain_abs = list(map(abs, data_grad_chain))
                
#         else:
#             data_grad_chain_abs_before = copy.deepcopy(data_grad_chain_abs)
#             # data_grad_chain_abs_now = copy.deepcopy(data_grad_chain_abs)
#             continue
#         break






#     # for px in range(1, 4097):
#     #     data_grad_chain = list(itertools.chain.from_iterable(data_grad[0][0]))
#     #     character_coordinate = character_search(data.data[0][0])
#     #     for i in range(4096):
#     #         if character_coordinate[i] == torch.Tensor([0]).to("cuda"):
#     #             data_grad_chain[i] = torch.Tensor([0]).to("cuda")
#     #     for j in max_index_list:
#     #         data_grad_chain[j] = torch.Tensor([0]).to("cuda")
#     #     grad_sorted = sorted(list(map(abs, data_grad_chain)), reverse=True)
#     #     data_grad_chain_abs = list(map(abs, data_grad_chain))
#     #     if px == 1:
#     #         all_blocks = []
#     #         for i in range(4):
#     #             for j in range(4):
#     #                 block = []
#     #                 for k in range(16):
#     #                     for l in range(16):
#     #                         block.append(data_grad_chain_abs[i * 1024 + j * 16 + k * 64 + l])
#     #                 block_sum = sum(block)
#     #                 all_blocks.append(block_sum)
            
#     #         block_max_id = all_blocks.index(max(all_blocks))
#     #         block_max_index = [block_max_id % 4, block_max_id // 4]
        
#     #         best_px = 0
#     #         for i in range(16):
#     #             for j in range(16):
#     #                 if best_px < data_grad_chain_abs[block_max_index[0] * 16 + block_max_index[1] * 1024 + i * 64 + j]:
#     #                     best_px = data_grad_chain_abs[block_max_index[0] * 16 + block_max_index[1] * 1024 + i * 64 + j]


        
#     #     data.requires_grad = False
#     #     cnt = 0
#     #     for k in grad_sorted:
#     #         cnt += 1
#     #         grad_max_index = data_grad_chain_abs.index(k)
            
#     #         if (data_grad_chain[grad_max_index] > 0 and data[0][0][grad_max_index // 64][grad_max_index - 64 * (grad_max_index // 64)] < 0) or (data_grad_chain[grad_max_index] < 0 and data[0][0][grad_max_index // 64][grad_max_index - 64 * (grad_max_index // 64)] > 0):
#     #             data[0][0][grad_max_index // 64][grad_max_index - 64 * (grad_max_index // 64)] *= -1
#     #             output = model(data)
#     #             if F.nll_loss(output, target) < loss_old:
#     #                 data[0][0][grad_max_index // 64][grad_max_index - 64 * (grad_max_index // 64)] *= -1
#     #                 continue
#     #             # image[0][0][grad_max_index // 64][grad_max_index - 64 * (grad_max_index // 64)] += ((image[0][0][grad_max_index // 64][grad_max_index - 64 * (grad_max_index // 64)] == torch.Tensor([-1]).to("cuda")) * 0.2 - torch.Tensor([0.1]).to("cuda")).squeeze(0)
#     #             else:
#     #                 max_index_list.append(grad_max_index)
#     #                 break
#     #     else:
#     #         stop = 1
#     #     perturbed_data, max_index, stop, output = attack(data, data_copy, data_grad, max_index, target, loss)
#     #     plt.xticks([], [])
#     #     plt.yticks([], [])
#     #     plt.imsave(dirname_pro + chr_lambda(init_pred.item()) + "/{}".format(count_list[init_pred.item()]) + "/" + "{}.png".format(px), perturbed_data.squeeze().detach().cpu().numpy(), cmap="gray")
#     #     print("px:{}".format(px))

#     #     pred = output.max(1, keepdim=True)[1]
#     #     data = perturbed_data
#     #     if pred.item() != target.item():
#     #         break
#     #     elif stop == 1:
#     #         break

#     #     data.requires_grad = True
#     #     # 損失を計算します
#     #     loss = F.nll_loss(output, target)
#     #     print("loss:{}".format(loss))
#     #     # 既存の勾配を全てゼロにします
#     #     model.zero_grad()
#     #     # 逆伝播させてモデルの勾配を計算します
#     #     loss.backward()
#     #     # データの勾配を取得します
#     #     data_grad = data.grad.data
#     return data, pred
        
#     # return image, max_index_list, stop, output

In [8]:
def attack(data, data_grad, target, loss_old, dirname_pro, chr, count):
    max_index_list = []
    px = 0
    
    data_grad_chain = list(itertools.chain.from_iterable(data_grad[0][0]))
    character_coordinate = character_search(data.data[0][0])
    for i in range(4096):
        if character_coordinate[i] == torch.Tensor([0]).to("cuda"):
            data_grad_chain[i] = torch.Tensor([0]).to("cuda")
    for j in max_index_list:
        data_grad_chain[j] = torch.Tensor([0]).to("cuda")
    data_grad_chain_abs = list(map(abs, data_grad_chain))
    all_blocks = []
    for i in range(4):
        for j in range(4):
            block = []
            for k in range(16):
                for l in range(16):
                    block.append(data_grad_chain_abs[i * 1024 + j * 16 + k * 64 + l])
            block_sum = sum(block)
            all_blocks.append(block_sum)
    
    # block_max1_id = all_blocks.index(max(all_blocks))
    # block_max1_index = [block_max1_id % 4, block_max1_id // 4]
    # all_blocks.remove(max(all_blocks))
    # block_max2_id = all_blocks.index(max(all_blocks))
    # block_max2_index = [block_max2_id % 4, block_max2_id // 4]

    block_max_index_list = []
    for i in sorted(all_blocks, reverse=True):
        if i== torch.Tensor([0]).to("cuda"):
            break
        block_max_id = all_blocks.index(i)
        block_max_index_list.append([block_max_id % 4, block_max_id // 4])

    for ind, block_max_index in enumerate(block_max_index_list):
        print("block:{}".format(ind))
        best_px = 0
        for i in range(16):
            for j in range(16):
                if best_px < data_grad_chain_abs[block_max_index[0] * 16 + block_max_index[1] * 1024 + i * 64 + j]:
                    best_id = block_max_index[0] * 16 + block_max_index[1] * 1024 + i * 64 + j
        
        if (data_grad_chain[best_id] > 0 and data[0][0][best_id // 64][best_id - 64 * (best_id // 64)] < 0) or (data_grad_chain[best_id] < 0 and data[0][0][best_id // 64][best_id - 64 * (best_id // 64)] > 0):
            data.requires_grad = False
            data[0][0][best_id // 64][best_id - 64 * (best_id // 64)] *= -1
            data.requires_grad = True
            output = model(data)
            loss = F.nll_loss(output, target)
            if loss < loss_old:
                data.requires_grad = False
                data[0][0][best_id // 64][best_id - 64 * (best_id // 64)] *= -1
            # image[0][0][grad_max_index // 64][grad_max_index - 64 * (grad_max_index // 64)] += ((image[0][0][grad_max_index // 64][grad_max_index - 64 * (grad_max_index // 64)] == torch.Tensor([-1]).to("cuda")) * 0.2 - torch.Tensor([0.1]).to("cuda")).squeeze(0)
            else:
                px += 1
                plt.xticks([], [])
                plt.yticks([], [])
                plt.imsave(dirname_pro + chr + "/{}".format(count) + "/" + "{}.png".format(px), data.squeeze().detach().cpu().numpy(), cmap="gray")
                print("px:{}".format(px))
                print("loss:{}".format(loss))
                pred = output.max(1, keepdim=True)[1]
                if pred.item() != target.item():
                    return data, pred
                loss_old = loss
                max_index_list.append(best_id)
                model.zero_grad()
                # 逆伝播させてモデルの勾配を計算します
                loss.backward()
                # データの勾配を取得します
                data_grad = data.grad.data
                data_grad_chain = list(itertools.chain.from_iterable(data_grad[0][0]))
                character_coordinate = character_search(data.data[0][0])
                for i in range(4096):
                    if character_coordinate[i] == torch.Tensor([0]).to("cuda"):
                        data_grad_chain[i] = torch.Tensor([0]).to("cuda")
                for j in max_index_list:
                    data_grad_chain[j] = torch.Tensor([0]).to("cuda")
                data_grad_chain_abs = list(map(abs, data_grad_chain))
        
        attack_list = [best_id]

        for p in attack_list:
            p_list = [p - 65, p - 64, p - 63, p - 1, p + 1, p + 63, p + 64, p + 65] 
            for i in range(8):
                p_max = -1
                for j in p_list:
                    if p_max < data_grad_chain_abs[j]:
                        p_max = data_grad_chain_abs[j]
                        p_max_id = j
                p_list.remove(p_max_id)
                
                if (data_grad_chain[p_max_id] > 0 and data[0][0][p_max_id // 64][p_max_id - 64 * (p_max_id // 64)] < 0) or (data_grad_chain[p_max_id] < 0 and data[0][0][p_max_id // 64][p_max_id - 64 * (p_max_id // 64)] > 0):
                    data.requires_grad = False
                    data[0][0][p_max_id // 64][p_max_id - 64 * (p_max_id // 64)] *= -1
                    data.requires_grad = True
                    output = model(data)
                    loss = F.nll_loss(output, target)
                    if loss < loss_old:
                        data.requires_grad = False
                        data[0][0][p_max_id // 64][p_max_id - 64 * (p_max_id // 64)] *= -1
                        continue
                    # image[0][0][grad_max_index // 64][grad_max_index - 64 * (grad_max_index // 64)] += ((image[0][0][grad_max_index // 64][grad_max_index - 64 * (grad_max_index // 64)] == torch.Tensor([-1]).to("cuda")) * 0.2 - torch.Tensor([0.1]).to("cuda")).squeeze(0)
                    px += 1
                    plt.xticks([], [])
                    plt.yticks([], [])
                    plt.imsave(dirname_pro + chr + "/{}".format(count) + "/" + "{}.png".format(px), data.squeeze().detach().cpu().numpy(), cmap="gray")
                    print("px:{}".format(px))
                    print("loss:{}".format(loss))
                    pred = output.max(1, keepdim=True)[1]
                    if pred.item() != target.item():
                        return data, pred
                    loss_old = loss
                    max_index_list.append(p_max_id)
                    attack_list.append(p_max_id)
                    model.zero_grad()
                    # 逆伝播させてモデルの勾配を計算します
                    loss.backward()
                    # データの勾配を取得します
                    data_grad = data.grad.data
                    data_grad_chain = list(itertools.chain.from_iterable(data_grad[0][0]))
                    character_coordinate = character_search(data.data[0][0])
                    for i in range(4096):
                        if character_coordinate[i] == torch.Tensor([0]).to("cuda"):
                            data_grad_chain[i] = torch.Tensor([0]).to("cuda")
                    for j in max_index_list:
                        data_grad_chain[j] = torch.Tensor([0]).to("cuda")
                    data_grad_chain_abs = list(map(abs, data_grad_chain))
    return data, pred

In [20]:
def test( model, device):
    dir = "test"
    chr_lambda = lambda a: chr(a + 65)
    # dirname_grad = "../PGD_alphabet_" + dir + "/grad/"
    # dirname_org = "../PGD_alphabet_" + dir + "/org/"
    # dirname_adv = "../PGD_alphabet_" + dir + "/adv/"
    # dirname_pro = "../PGD_alphabet_" + dir + "/progress/"
    dirname_grad = "../PGD_alphabet_" + dir + "_upd2" + "/grad/"
    dirname_org = "../PGD_alphabet_" + dir + "_upd2" + "/org/"
    dirname_adv = "../PGD_alphabet_" + dir + "_upd2" + "/adv/"
    dirname_pro = "../PGD_alphabet_" + dir + "_upd2" + "/progress/"
    for c in [chr(i) for i in range(65, 65+26)]:
        os.makedirs(dirname_grad + c, exist_ok=True)
        os.makedirs(dirname_org + c, exist_ok=True)
        os.makedirs(dirname_adv + c, exist_ok=True)
        os.makedirs(dirname_pro + c, exist_ok=True)
        for d in [chr(i) for i in range(65, 65+26)]:
            os.makedirs(dirname_adv + c + "/" + c + "→" + d, exist_ok=True)
        for e in range(1, 401):
            os.makedirs(dirname_pro + c + "/{}".format(e), exist_ok=True)
    
    datas = torch.from_numpy(dataset_64.X_test.astype(np.float32)).clone()
    labels = torch.from_numpy(dataset_64.y_test.astype(np.float32)).clone()

    # 精度カウンター
    correct = 0
    # count = 0
    count_list = [0] * 26

    adv_examples = []
    # i=0
    

    # テスト用データセット内の全てのサンプルをループします
    for data, target in zip(datas, labels):
        
        # i+=1
        # if i==150:
        #     break
        # データとラベルをデバイス（CPUもしくはGPU）に送信します
        data, target = data.to(device), target.type(torch.LongTensor).to(device)
        # テンソルの requires_grad 属性を設定します。攻撃者にとっては重要な設定です。
        data.requires_grad = True

        # データをモデルに順伝播させます
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1] # 最大の確率のインデックスを取得します。

        # 最初から予測が間違っている場合、攻撃する必要がないため次のイテレーションに進みます。
        
        if init_pred.item() != target.item() or init_pred.item() != 8:
            continue

        data_copy = data.detach().clone()
        # character_coordinate = character_search(data_copy.data[0][0])
        
        count_list[init_pred.item()] += 1
        
    
        # 損失を計算します
        loss = F.nll_loss(output, target)
        print("loss:{}".format(loss))
        print(type(target))
        break
        # 既存の勾配を全てゼロにします
        model.zero_grad()
        # 逆伝播させてモデルの勾配を計算します
        loss.backward()
        # データの勾配を取得します
        data_grad = data.grad.data
        
        max_index = []
        #勾配のヒートマップ
        grad_map = data_grad.squeeze().detach().cpu().numpy()
        grad_map_abs = np.abs(grad_map)
        plt.xticks([], [])
        plt.yticks([], [])
        plt.imsave(dirname_grad + chr_lambda(init_pred.item()) + "/{}.png".format(count_list[init_pred.item()]), grad_map_abs, cmap="Reds")

        perturbed_data, pred = attack(data, data_grad, target, loss, dirname_pro, chr_lambda(init_pred.item()), count_list[init_pred.item()])
        
        final_pred = pred

        org = data_copy.squeeze().detach().cpu().numpy()
        adv = perturbed_data.squeeze().detach().cpu().numpy()
        

        #各条件を満たす画像の保存
        plt.xticks([], [])
        plt.yticks([], [])
        print(dirname_org + chr_lambda(init_pred.item()) + "/{}.png".format(count_list[init_pred.item()]))
        plt.imsave(dirname_org + chr_lambda(init_pred.item()) + "/{}.png".format(count_list[init_pred.item()]), org, cmap="gray")
        
        os.makedirs(dirname_adv + chr_lambda(init_pred.item()) + "/" + chr_lambda(init_pred.item()) + "→" + chr_lambda(final_pred.item()) + "/", exist_ok=True)
        plt.xticks([], [])
        plt.yticks([], [])
        plt.imsave(dirname_adv + chr_lambda(init_pred.item()) + "/" + chr_lambda(init_pred.item()) + "→" + chr_lambda(final_pred.item()) + "/{}.png".format(count_list[init_pred.item()]), adv, cmap="gray")

        if count_list[init_pred.item()] == 50:
            break

        if (final_pred.item() != target.item()):
            # あとで可視化するために敵対的サンプルのうちいくつかを保存
            if len(adv_examples) < 25:
                adv_examples.append( (init_pred.item(), final_pred.item(), org, adv))
                # if len(adv_examples) == 25:
                #     break


        # # epsilonごとの最終的な精度を算出
        # final_acc = correct/float(len(test_loader))
        # print("Test Accuracy = {} / {} = {}".format(correct, len(test_loader), final_acc))

        # # 精度と敵対的サンプルを返却
        # return final_acc, adv_examples

In [21]:
accuracies = []
examples = []

# 各epsilonごとにテストを実行
acc, ex = test(model, device)
accuracies.append(acc)
examples.append(ex)

loss:0.00016175392374861985
<class 'torch.Tensor'>


TypeError: cannot unpack non-iterable NoneType object

In [14]:
cnt = 0
plt.figure(figsize=(8,10))

for j in range(len(examples[0])):
    cnt += 1
    plt.subplot(5,5,cnt)
    plt.xticks([], [])
    plt.yticks([], [])
    orig,adv, org_ex, adv_ex = examples[0][j]
    plt.title("{}".format(orig), color="white")
    plt.imshow(org_ex, cmap="gray")
plt.tight_layout()
plt.show()

IndexError: list index out of range

<Figure size 576x720 with 0 Axes>

In [ ]:
cnt = 0
plt.figure(figsize=(8,10))

for j in range(len(examples[0])):
    cnt += 1
    plt.subplot(5,5,cnt)
    plt.xticks([], [])
    plt.yticks([], [])
    orig, adv, org_ex, adv_ex = examples[0][j]
    plt.title("{} -> {}".format(orig, adv), color="white")
    plt.imshow(adv_ex, cmap="gray")
plt.tight_layout()
plt.show()